In [43]:
import json
import requests
import yaml
import pandas as pd
from pandas import concat
from datetime import datetime
from pandas import json_normalize

In [44]:
### Get list of commune
# https://www.bfs.admin.ch/bfs/en/home/statistics/catalogues-databases/tables.assetdetail.16484444.html
commune_df = pd.read_csv("./../../data/raw/RegionalPortraits21.03.01.csv", sep=';', low_memory=False)
commune_df = commune_df[['REGION', 'VALUE']]

# for row in commune_df.iterrows():
#     region = row[1]['REGION'] # Region 
#     population = row[1]['VALUE'] # Population
#     print(f'Region = `{region}`, Population = `{population}`')
    
commune_df

,REGION,VALUE
0,Aeugst am Albis,1981
1,Affoltern am Albis,12303
2,Bonstetten,5572
3,Hausen am Albis,3751
4,Hedingen,3778
...,...,...
2167,Vendlincourt,559
2168,Basse-Allaine,1238
2169,Clos du Doubs,1263
2170,Haute-Ajoie,1085


In [55]:
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

# actual api key
config = yaml.load(open("./../../credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = config["google_map_key"]

# get method of requests module, return response object
row_num = 0
request_num = 0
for row in commune_df.iterrows():
    row_num += 1
    # if row_num in range(1, 207): # start from 1
    #     continue
    
    region = row[1]['REGION'].replace('/', '__') # Region | Commune
    
    # text string on which to search
    query = f'supermarkets near {region}, Switzerland'
    
    df = pd.DataFrame()
    next_page_token = None
    while True:
        request_num += 1
        req = requests.get(url + "query=" + query + "&key=" + gmaps_key \
           #+ "&region=ch&type=supermarket" \
           + ('' if next_page_token == None else f'&pageToken={next_page_token}'))

        # json method of response object: json format data -> python format data
        places_json = req.json()
        print(places_json)

        # process request response
        for result in places_json['results']:
            df = pd.concat([df, json_normalize(result)], ignore_index=True)
            #df.loc[df.astype(str).drop_duplicates().index] # otherwise will be TypeError: unhashable type: 'list'

        # inform an request has been processed
        # print(f"{i} - A request has been processed at", datetime.now().strftime("%H:%M:%S"))

        if ('next_page_token' in places_json.keys()):
            next_page_token = places_json['next_page_token']
        else:
            #df = df.loc[df.astype(str).drop_duplicates(subset=['place_id']).index]
            df = df.drop_duplicates(subset=['place_id'])
            df.to_csv(f"./../../data/raw/gmaps_data_{region}.csv")
            print(f"{row_num} - {request_num} - gmaps_data_{region}.csv created at", datetime.now().strftime("%H:%M:%S"))
            break
print('done!')

{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
1 - 1 - gmaps_data_Aeugst am Albis.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
2 - 2 - gmaps_data_Affoltern am Albis.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
3 - 3 - gmaps_data_Bonstetten.csv created at 20:51:16
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
4 - 4 - gmaps_data_Hausen am Albis.csv created at 20:51:17
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
5 - 5 - gmaps_data_Hedingen.csv created at 20:51:17
{'error_message': 'The provided API key is expired.', 'html_attributions': [], 'results': [], 'status': 

KeyboardInterrupt: 

In [41]:
### Merging all data of Commune into one raw table
df = pd.DataFrame()
for row in commune_df.iterrows():
    region = row[1]['REGION'].replace('/', '__') # Region | Commune
    df = pd.concat([df, pd.read_csv(f"./../../data/raw/gmaps_data_{region}.csv", low_memory=False)])
df = df.drop_duplicates(subset=['place_id'])
df.to_csv("./../../data/raw/gmaps_data.csv")
df

,Unnamed: 0,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,...,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,price_level,permanently_closed
0,0,OPERATIONAL,"Allmendstrasse 5, 8914 Aeugst am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Volg,"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJoetcG74HkEcR5_9mPkmNtSI,4.4,...,8.486222,47.269277,8.487494,47.266577,8.484794,True,7F9P+6C Aeugst am Albis,8FVC7F9P+6C,NaN,NaN
1,1,OPERATIONAL,"Oberdorfstrasse 4, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Migros Supermarkt,"[{'height': 1920, 'html_attributions': ['<a hr...",ChIJiUOy2zkGkEcRy22nDwkqAks,4.2,...,8.453773,47.279066,8.455123,47.276367,8.452424,True,7FH3+4G Affoltern am Albis,8FVC7FH3+4G,NaN,NaN
2,2,OPERATIONAL,"Büelstrasse 15, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Coop Supermarkt Affoltern am Albis,"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJk6WFSkcGkEcRwsvlTASqby8,4.1,...,8.445676,47.275559,8.447437,47.272859,8.444737,True,7CFW+G7 Affoltern am Albis,8FVC7CFW+G7,NaN,NaN
3,3,OPERATIONAL,"Centralweg 3, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 1200, 'html_attributions': ['<a hr...",ChIJW4AoZTcGkEcRZ-7-G4qYMg0,4.1,...,8.453030,47.279424,8.454497,47.276724,8.451797,True,7FH3+66 Affoltern am Albis,8FVC7FH3+66,NaN,NaN
4,4,OPERATIONAL,"Industriestrasse 5, 8910 Affoltern am Albis",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,ALDI SUISSE,"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJLzM1Oh8HkEcRwBmfpSnSZx8,4.1,...,8.447015,47.274182,8.448102,47.271483,8.445402,True,7CFW+3R Affoltern am Albis,8FVC7CFW+3R,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,6,OPERATIONAL,"Dorfstrasse 1, 8197 Rafz",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Volg Rafz,"[{'height': 1728, 'html_attributions': ['<a hr...",ChIJadCMQuZ5kEcRlMSlbbmuFzg,4.4,...,8.536855,47.613674,8.538179,47.610974,8.535479,True,JG6P+WM Rafz,8FVCJG6P+WM,NaN,NaN
12,12,OPERATIONAL,"Industriestraße 36, 79807 Lottstetten, Germany",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl,"[{'height': 2988, 'html_attributions': ['<a hr...",ChIJoV16IE54kEcRwf420aRhKKc,4.1,...,8.578581,47.632467,8.580241,47.629767,8.577541,True,"JHJH+CC Lottstetten, Germany",8FVCJHJH+CC,NaN,NaN
14,14,OPERATIONAL,"Emmerstrasse 12, 8192 Glattfelden",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Coop Supermarkt Glattfelden,"[{'height': 1960, 'html_attributions': ['<a hr...",ChIJYQBygs12kEcRFEG25WHmtks,4.1,...,8.502778,47.558039,8.504261,47.555339,8.501562,True,HG43+P4 Glattfelden,8FVCHG43+P4,NaN,NaN
15,15,OPERATIONAL,"Schaffhauserstrasse 39, 8193 Eglisau",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,migrolino Eglisau,"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJPSOMLDF3kEcR3XUwsuQU9Hs,4.3,...,8.516040,47.582226,8.517641,47.579526,8.514942,True,HGJ8+8C Eglisau,8FVCHGJ8+8C,NaN,NaN


In [42]:
original_df = pd.read_csv("./../../data/raw/gmaps_data.csv", low_memory=False)
df = original_df[['place_id', 'formatted_address', 'name', 'rating', 'types', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng', 'plus_code.compound_code']].copy()

df['canton'] = df.apply(lambda row: row.formatted_address.split(',')[1][6:], axis=1)
df['brand'] = df['name'].apply(lambda x: 'Migros' if ('Migros' in x) or ('migros' in x) else x)
df.to_csv("./../../data/processed/gmaps_data_all_brands.csv")
df[df['brand'] == 'Migros'].to_csv("./../../data/processed/gmaps_data_migros_only.csv")
df[df['brand'] != 'Migros'].to_csv("./../../data/processed/gmaps_data_other_brands.csv")
df

,place_id,formatted_address,name,rating,types,user_ratings_total,geometry.location.lat,geometry.location.lng,plus_code.compound_code,canton,brand
0,ChIJoetcG74HkEcR5_9mPkmNtSI,"Allmendstrasse 5, 8914 Aeugst am Albis",Volg,4.4,"['supermarket', 'grocery_or_supermarket', 'sto...",47.0,47.267973,8.486222,7F9P+6C Aeugst am Albis,Aeugst am Albis,Volg
1,ChIJiUOy2zkGkEcRy22nDwkqAks,"Oberdorfstrasse 4, 8910 Affoltern am Albis",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'sto...",578.0,47.277794,8.453773,7FH3+4G Affoltern am Albis,Affoltern am Albis,Migros
2,ChIJk6WFSkcGkEcRwsvlTASqby8,"Büelstrasse 15, 8910 Affoltern am Albis",Coop Supermarkt Affoltern am Albis,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",1179.0,47.273871,8.445676,7CFW+G7 Affoltern am Albis,Affoltern am Albis,Coop Supermarkt Affoltern am Albis
3,ChIJW4AoZTcGkEcRZ-7-G4qYMg0,"Centralweg 3, 8910 Affoltern am Albis",Lidl Schweiz,4.1,"['supermarket', 'convenience_store', 'grocery_...",292.0,47.278007,8.453030,7FH3+66 Affoltern am Albis,Affoltern am Albis,Lidl Schweiz
4,ChIJLzM1Oh8HkEcRwBmfpSnSZx8,"Industriestrasse 5, 8910 Affoltern am Albis",ALDI SUISSE,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",292.0,47.272725,8.447015,7CFW+3R Affoltern am Albis,Affoltern am Albis,ALDI SUISSE
...,...,...,...,...,...,...,...,...,...,...,...
159,ChIJadCMQuZ5kEcRlMSlbbmuFzg,"Dorfstrasse 1, 8197 Rafz",Volg Rafz,4.4,"['supermarket', 'grocery_or_supermarket', 'sto...",146.0,47.612332,8.536855,JG6P+WM Rafz,Rafz,Volg Rafz
160,ChIJoV16IE54kEcRwf420aRhKKc,"Industriestraße 36, 79807 Lottstetten, Germany",Lidl,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",1019.0,47.631005,8.578581,"JHJH+CC Lottstetten, Germany",Lottstetten,Lidl
161,ChIJYQBygs12kEcRFEG25WHmtks,"Emmerstrasse 12, 8192 Glattfelden",Coop Supermarkt Glattfelden,4.1,"['supermarket', 'grocery_or_supermarket', 'sto...",248.0,47.556828,8.502778,HG43+P4 Glattfelden,Glattfelden,Coop Supermarkt Glattfelden
162,ChIJPSOMLDF3kEcR3XUwsuQU9Hs,"Schaffhauserstrasse 39, 8193 Eglisau",migrolino Eglisau,4.3,"['supermarket', 'grocery_or_supermarket', 'sto...",123.0,47.580756,8.516040,HGJ8+8C Eglisau,Eglisau,migrolino Eglisau


In [26]:


grouped_df = df.groupby(by=['canton', 'name']).agg({
    'rating': ['mean', 'median', 'count'],
    'user_ratings_total': 'sum',
    'geometry.location.lat': 'count'
}).rename(columns={
    'rating_count': 'count'
}).reset_index()

grouped_df[grouped_df['name'] == 'Migros']
grouped_df

#df.groupby(["item", "color"])["id"].count().reset_index(name="count")

canton                                       name rating         \
                                                                  mean median   
0          Jestetten                                       ALDI   4.00   4.00   
1        Lottstetten                Barış Turkish Grocery Store   4.70   4.70   
2        Lottstetten                                       Lidl   4.10   4.10   
3            Adlikon  BP Service Weinland West Express Shopping   4.20   4.20   
4           Adliswil                                ALDI SUISSE   4.35   4.35   
..               ...                                        ...    ...    ...   
145           Zürich      migrolino Zürich Birmensdorferstrasse   4.30   4.30   
146      erstrasse 1            Coop Supermarkt Uitikon Waldegg   4.40   4.40   
147    erstrasse 295                                       Lidl   4.10   4.10   
148  hübelstrasse 65                          Migros Supermarkt   4.30   4.30   
149      rt Shopping               Coop Pronto Zürich Flughafen   4.00   4.00   

          user_ratings_total geometry.location.lat  
    count                sum                 count  
0    4509          4152789.0                  4509  
1       1               72.0                     1  
2       1             1019.0                     1  
3       1              170.0                     1  
4       2              238.0                     2  
..    ...                ...                   ...  
145     1               26.0                     1  
146     1              126.0                     1  
147     1              310.0                     1  
148     2             1624.0                     2  
149     1               78.0                     1  

[150 rows x 7 columns]

In [48]:
df = pd.read_csv("./../../data/processed/migros_stores_data.csv", low_memory=False)
df = df.drop_duplicates(subset=['unique_id'])
df

,Unnamed: 0,unique_id,osmid,element_type,addr:housenumber,addr:postcode,addr:street,brand,brand:wikidata,cash_withdrawal,...,end_date,addr:state,loc_name,wikidata,payment:telephone_cards,kids_area,operator:type,toilets,toilets:access,name:it
0,0,node/355287954,355287954,node,32,3963.0,Route du Rawyl,Migros,Q680727,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,node/357276931,357276931,node,NaN,NaN,NaN,Migros,Q680727,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,node/357299653,357299653,node,NaN,NaN,NaN,Migros,Q680727,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,node/454026916,454026916,node,NaN,NaN,NaN,Migros,Q680727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,node/886099069,886099069,node,NaN,NaN,NaN,Migros,Q680727,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849,1849,node/3758933750,3758933750,node,NaN,6900.0,Viale Castagnola,Migros,Q680727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1850,1850,node/4603794691,4603794691,node,NaN,NaN,NaN,NaN,NaN,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Migros
1851,1851,node/5046260927,5046260927,node,20,6834.0,Viale Serfontana,Migros,Q680727,postfinance_card,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1852,1852,way/113684683,113684683,way,40,6950.0,Via Luigi Canonica,Migros,Q680727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
df = pd.read_csv("./../../data/raw/supermarkets_new.csv", low_memory=False)
df = df.drop_duplicates(subset=['place_id'])
df[df['name'].str.contains("Migros")]

,Unnamed: 0,formatted_address,name,place_id,rating,user_ratings_total,geometry.location.lat,geometry.location.lng,opening_hours.open_now,permanently_closed,Street,commune,country,store_name
3,6,"Oberdorfstrasse 4, 8910 Affoltern am Albis, Sw...",Migros Supermarkt,ChIJiUOy2zkGkEcRy22nDwkqAks,4.2,578.0,47.277794,8.453773,True,NaN,Oberdorfstrasse 4,8910 Affoltern am Albis,Switzerland,migro
10,37,"Schachenstrasse 115, 8906 Bonstetten, Switzerland",Migros Partner,ChIJH9iPzvsIkEcRGKrRIuBujB8,4.4,124.0,47.325319,8.472515,True,NaN,Schachenstrasse 115,8906 Bonstetten,Switzerland,migro
12,44,"Hintere Grundstrasse 2, 8135 Langnau am Albis,...",Migros Supermarkt,ChIJy4I_EeGnmkcRGRIhApH6DXI,4.2,132.0,47.291338,8.537833,True,NaN,Hintere Grundstrasse 2,8135 Langnau am Albis,Switzerland,migro
17,57,"Chamerstrasse 12, 6331 Hünenberg, Switzerland",Migros Supermarkt ab 24.11.22,ChIJW6acDaQBkEcRfud3Va_PkRI,0.0,0.0,47.175089,8.427163,NaN,NaN,Chamerstrasse 12,6331 Hünenberg,Switzerland,migro
19,61,"Maihölzli 5, 6331 Hünenberg, Switzerland",Migros-Supermarkt - Hünenberg Dorf,ChIJx-xVC2QBkEcRIOsffv2bKVM,0.0,0.0,47.175046,8.427322,True,NaN,Maihölzli 5,6331 Hünenberg,Switzerland,migro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2718,10600,"Rte de Saint-Julien 176, 1228 Plan-les-Ouates,...",Supermarché Migros Plan-les-Ouates,ChIJuVp8MQx7jEcRODwRuVZLSjE,4.0,716.0,46.167514,6.115288,True,NaN,Rte de Saint-Julien 176,1228 Plan-les-Ouates,Switzerland,migro
2724,10627,"Arcenter, Rte de Peney 2-4, 1214 Vernier, Swit...",Migros,ChIJLT3osUZjjEcRz7dfie0McU0,4.2,317.0,46.216978,6.083779,True,NaN,Arcenter,Rte de Peney 2-4,1214 Vernier,migro
2726,10634,"Rte de Vernier 184, 1214 Vernier, Switzerland",Migros Outlet,ChIJB5a0JUNjjEcR1k-fMSJkhUo,4.0,418.0,46.217051,6.087175,True,NaN,Rte de Vernier 184,1214 Vernier,Switzerland,migro
2731,10658,"Fbg Saint-Germain 2, 2900 Porrentruy, Switzerland",Supermarché Migros,ChIJPyxh7Q_3kUcRmJjHf-_q-qc,4.1,546.0,47.416548,7.078071,True,NaN,Fbg Saint-Germain 2,2900 Porrentruy,Switzerland,migro


In [61]:
df = pd.read_csv("./../../data/processed/gmaps_data.csv", low_memory=False)
df = df.drop_duplicates(subset=['formatted_address'])
df[df['name'].str.contains("Migros")]

,Unnamed: 0,formatted_address,name,rating,types,user_ratings_total,geometry.location.lat,geometry.location.lng,plus_code.compound_code,canton,brand
1,1,"Oberdorfstrasse 4, 8910 Affoltern am Albis",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'sto...",578.0,47.277794,8.453773,7FH3+4G Affoltern am Albis,Affoltern am Albis,Migros
8,8,"Albisstrasse 10, 8134 Adliswil",Migros Supermarkt,4.3,"['grocery_or_supermarket', 'supermarket', 'sto...",877.0,47.311111,8.525278,8G6G+C4 Adliswil,Adliswil,Migros
9,9,"Hintere Grundstrasse 2, 8135 Langnau am Albis",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'sto...",132.0,47.291338,8.537833,7GRQ+G4 Langnau am Albis,Langnau am Albis,Migros
12,12,"Zürcherstrasse 4, 8803 Rüschlikon",Migros Supermarkt,4.4,"['grocery_or_supermarket', 'supermarket', 'sto...",696.0,47.300096,8.548924,8G2X+2H Rüschlikon,Rüschlikon,Migros
17,17,"Alte Landstrasse 130, 8800 Thalwil",Migros Supermarkt,4.3,"['grocery_or_supermarket', 'supermarket', 'sto...",557.0,47.294187,8.564082,7HV7+MJ Thalwil,Thalwil,Migros
29,29,"Jonas-Furrer-Strasse 21, 8046 Zürich",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'foo...",900.0,47.420064,8.508500,CGC5+2C Zürich,Zürich,Migros
35,35,"Schachenstrasse 115, 8906 Bonstetten",Migros Partner,4.4,"['supermarket', 'bakery', 'grocery_or_supermar...",124.0,47.325319,8.472515,8FGC+4X Bonstetten,Bonstetten,Migros
38,38,"Zürcherstrasse 15, 8903 Birmensdorf",Migros Supermarkt,4.3,"['grocery_or_supermarket', 'supermarket', 'foo...",176.0,47.354294,8.436778,9C3P+PP Birmensdorf,Birmensdorf,Migros
58,58,"Alte Landstrasse 24, 8810 Horgen",Migros Supermarkt,4.2,"['grocery_or_supermarket', 'supermarket', 'sto...",543.0,47.260458,8.597384,7H6W+5X Horgen,Horgen,Migros
84,84,"Bahnhofstrasse 5, 6340 Baar",Migros Supermarkt,4.3,"['grocery_or_supermarket', 'supermarket', 'foo...",319.0,47.195612,8.524921,5GWG+62 Baar,Baar,Migros
